In [102]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [103]:
# https://www.statsmodels.org/stable/index.html
import statsmodels.api as sm

In [104]:
import warnings
warnings.filterwarnings("ignore")

In [105]:
from functools import partial

In [106]:
from dotenv import load_dotenv

from pathlib import Path

env_path = Path("../../.env-live")

if env_path.exists():
    print('envs Loaded')
    load_dotenv(dotenv_path=env_path)
from jrjModelRegistry.jrjModelRegistry import registerAJrjModel

envs Loaded


In [107]:
sallaryMisDf = pd.read_excel("https://www.dropbox.com/scl/fi/zt2vtwhpz8ndblsxqdqx1/Salary_MIS.xlsx?rlkey=2uk6m7m9w90isv6zsynhhhpyv&st=gxumjns5&dl=1")

In [108]:
sallaryMisDf

,Salary,GPA,MIS,Statistics
0,72,3.53,1,0
1,66,2.86,1,0
2,72,3.69,0,0
3,63,3.24,0,0
4,65,3.21,0,0
...,...,...,...,...
115,66,3.27,0,0
116,63,2.86,1,0
117,78,3.04,1,1
118,64,2.99,0,0


In [109]:
sallaryMisDf.shape

(120, 4)

In [110]:
sallaryMisDf.describe()

,Salary,GPA,MIS,Statistics
count,120.000000,120.000000,120.000000,120.000000
mean,69.875000,3.242750,0.316667,0.341667
std,6.594577,0.493834,0.467127,0.476257
min,53.000000,2.410000,0.000000,0.000000
25%,65.750000,2.805000,0.000000,0.000000
50%,70.000000,3.280000,0.000000,0.000000
75%,73.250000,3.692500,1.000000,1.000000
max,88.000000,3.980000,1.000000,1.000000


In [111]:
sallaryMisDf.shape

(120, 4)

In [112]:
sm.add_constant(sallaryMisDf[["GPA", "MIS", "Statistics"]])

,const,GPA,MIS,Statistics
0,1.0,3.53,1,0
1,1.0,2.86,1,0
2,1.0,3.69,0,0
3,1.0,3.24,0,0
4,1.0,3.21,0,0
...,...,...,...,...
115,1.0,3.27,0,0
116,1.0,2.86,1,0
117,1.0,3.04,1,1
118,1.0,2.99,0,0


In [113]:
salaryRegModel1 = sm.OLS(
  sallaryMisDf["Salary"],
  sm.add_constant(sallaryMisDf[["GPA", "MIS", "Statistics"]])
)

In [114]:
salaryRegModel1Fit = salaryRegModel1.fit()

In [115]:
print(salaryRegModel1Fit.summary())

                            OLS Regression Results                            
Dep. Variable:                 Salary   R-squared:                       0.795
Model:                            OLS   Adj. R-squared:                  0.790
Method:                 Least Squares   F-statistic:                     150.3
Date:                Wed, 04 Jun 2025   Prob (F-statistic):           8.35e-40
Time:                        02:14:24   Log-Likelihood:                -300.92
No. Observations:                 120   AIC:                             609.8
Df Residuals:                     116   BIC:                             621.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         44.0072      1.860     23.662      0.0

In [116]:
sallaryMisDf

,Salary,GPA,MIS,Statistics
0,72,3.53,1,0
1,66,2.86,1,0
2,72,3.69,0,0
3,63,3.24,0,0
4,65,3.21,0,0
...,...,...,...,...
115,66,3.27,0,0
116,63,2.86,1,0
117,78,3.04,1,1
118,64,2.99,0,0


In [117]:
salaryRegModel1SampleData = {
    "GPA": [3.53, 3.04, 3.65],
    "MIS": [1, 1, 0],
    "Statistics": [0,1, 0]
}

In [118]:
def salaryRegModel1Transformer(dataForTransfer = None):
    import pandas as pd
    import statsmodels.api as sm
    dfTransformer = pd.DataFrame(dataForTransfer)
    dfTransformer = sm.add_constant(dfTransformer[['GPA', 'MIS', 'Statistics']])
    return dfTransformer

In [119]:
def salaryRegModel1MainPredictor(self, transformedData):
    return self.predict(transformedData)

In [120]:
salaryRegModel1Fit.mainPredictor = partial(salaryRegModel1MainPredictor, salaryRegModel1Fit)

In [121]:
salaryRegModel1Fit.transformer = salaryRegModel1Transformer

In [122]:
test1 = salaryRegModel1Fit.transformer(salaryRegModel1SampleData)
test1
# salaryRegModel1Fit.mainPredictor(test1)

,const,GPA,MIS,Statistics
0,1.0,3.53,1,0
1,1.0,3.04,1,1
2,1.0,3.65,0,0


In [123]:
salaryRegModel1Fit.mainPredictor(test1)

0    73.992468
1    77.478273
2    68.180124
dtype: float64

In [124]:
salaryRegModel1FitMetadata = {
    "modelName":f"salaryRegModel1Fit",
    "version":"1.0.1",
    "params": salaryRegModel1Fit.params.to_dict(),
    "score": float(salaryRegModel1Fit.rsquared),
    "modelLibraray": 'sm.OLS',
    "libraryMetadata": {
        "pvalues": salaryRegModel1Fit.pvalues.to_dict(),
        "r_squared": float(salaryRegModel1Fit.rsquared),
        "adj_r_squared": float(salaryRegModel1Fit.rsquared_adj)
    },

    "sampleData": {
        "dataForTransfer": salaryRegModel1SampleData
    }
}
salaryRegModel1FitMetadata

{'modelName': 'salaryRegModel1Fit',
 'version': '1.0.1',
 'params': {'const': 44.00724634057489,
  'GPA': 6.62270615035616,
  'MIS': 6.607068893822084,
  'Statistics': 6.730931143358017},
 'score': 0.7953924097617404,
 'modelLibraray': 'sm.OLS',
 'libraryMetadata': {'pvalues': {'const': 3.2704010980896227e-46,
   'GPA': 3.077793077406486e-21,
   'MIS': 6.058766938187377e-20,
   'Statistics': 1.2379781357348497e-20},
  'r_squared': 0.7953924097617404,
  'adj_r_squared': 0.7901008341521303},
 'sampleData': {'dataForTransfer': {'GPA': [3.53, 3.04, 3.65],
   'MIS': [1, 1, 0],
   'Statistics': [0, 1, 0]}}}

In [125]:
registerAJrjModel(salaryRegModel1Fit, salaryRegModel1FitMetadata)

✅ Uploaded encrypted ZIP to s3://jasonjafariml/salaryRegModel1Fit__1.0.1.pkl.zip


{'_id': '683fe44235856819138785c5',
 'modelName': 'salaryRegModel1Fit',
 'version': '1.0.1',
 'params': {'const': 44.00724634057489,
  'GPA': 6.62270615035616,
  'MIS': 6.607068893822084,
  'Statistics': 6.730931143358017},
 'score': 0.7953924097617404,
 'modelLibraray': 'sm.OLS',
 'libraryMetadata': {'pvalues': {'const': 3.2704010980896227e-46,
   'GPA': 3.077793077406486e-21,
   'MIS': 6.058766938187377e-20,
   'Statistics': 1.2379781357348497e-20},
  'r_squared': 0.7953924097617404,
  'adj_r_squared': 0.7901008341521303},
 'sampleData': {'dataForTransfer': {'GPA': [3.53, 3.04, 3.65],
   'MIS': [1, 1, 0],
   'Statistics': [0, 1, 0]}},
 'modelType': 'model',
 'keepLastOnly': False,
 'modelSizeBytes': 20655,
 'zippedModelSizeBytes': 7989,
 's3Url': 'jasonjafariml/salaryRegModel1Fit__1.0.1.pkl.zip',
 'createdAt': '2025-06-04T06:14:26.105Z',
 'updatedAt': '2025-06-04T06:14:26.105Z'}

In [126]:
sallaryMisDf

,Salary,GPA,MIS,Statistics
0,72,3.53,1,0
1,66,2.86,1,0
2,72,3.69,0,0
3,63,3.24,0,0
4,65,3.21,0,0
...,...,...,...,...
115,66,3.27,0,0
116,63,2.86,1,0
117,78,3.04,1,1
118,64,2.99,0,0


In [127]:
sallaryMisDf["misXStatistics"] = sallaryMisDf["MIS"] * sallaryMisDf["Statistics"]
sallaryMisDf['misXStatistics1'] = sallaryMisDf.apply(lambda row: row['MIS'] * row['Statistics'], axis=1)

sallaryMisDf

,Salary,GPA,MIS,Statistics,misXStatistics,misXStatistics1
0,72,3.53,1,0,0,0.0
1,66,2.86,1,0,0,0.0
2,72,3.69,0,0,0,0.0
3,63,3.24,0,0,0,0.0
4,65,3.21,0,0,0,0.0
...,...,...,...,...,...,...
115,66,3.27,0,0,0,0.0
116,63,2.86,1,0,0,0.0
117,78,3.04,1,1,1,1.0
118,64,2.99,0,0,0,0.0


In [128]:
salaryRegModel2 = sm.OLS(
  sallaryMisDf["Salary"],
  sm.add_constant(
      sallaryMisDf[[
          "GPA",
          "MIS",
          "Statistics",
          "misXStatistics1"
      ]]
  )
)

In [129]:
salaryRegModel2Fit = salaryRegModel2.fit()

In [130]:
print(salaryRegModel2Fit.summary())

                            OLS Regression Results                            
Dep. Variable:                 Salary   R-squared:                       0.810
Model:                            OLS   Adj. R-squared:                  0.803
Method:                 Least Squares   F-statistic:                     122.2
Date:                Wed, 04 Jun 2025   Prob (F-statistic):           1.87e-40
Time:                        02:14:26   Log-Likelihood:                -296.63
No. Observations:                 120   AIC:                             603.3
Df Residuals:                     115   BIC:                             617.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              44.0993      1.803     

In [131]:
salaryRegModel2SampleData = salaryRegModel1SampleData

In [132]:
def salaryRegModel2Transformer(dataForTransfer = None):
    import pandas as pd
    import statsmodels.api as sm
    dfTransformer = pd.DataFrame(dataForTransfer)
    dfTransformer['misXStatistics1'] = dfTransformer.apply(lambda row: row['MIS'] * row['Statistics'], axis=1)
    dfTransformer = sm.add_constant(dfTransformer[['GPA', 'MIS', 'Statistics', 'misXStatistics1']])
    return dfTransformer

In [133]:
salaryRegModel2Fit.transformer = salaryRegModel2Transformer

In [134]:
salaryRegModel2Fit.mainPredictor = partial(salaryRegModel1MainPredictor, salaryRegModel2Fit)

In [135]:
test2 = salaryRegModel2Fit.transformer(salaryRegModel2SampleData)
test2

,const,GPA,MIS,Statistics,misXStatistics1
0,1.0,3.53,1,0,0.0
1,1.0,3.04,1,1,1.0
2,1.0,3.65,0,0,0.0


In [136]:
salaryRegModel2Fit.mainPredictor(test2)

0    73.113927
1    78.852012
2    68.594218
dtype: float64

In [137]:
salaryRegModel2FitMetadata = {
    "modelName":f"salaryRegModel2Fit",
    "version":"1.0.1",
    "params": salaryRegModel2Fit.params.to_dict(),
    "score": float(salaryRegModel2Fit.rsquared),
    "modelLibraray": 'sm.OLS',
    "libraryMetadata": {
        "pvalues": salaryRegModel2Fit.pvalues.to_dict(),
        "r_squared": float(salaryRegModel2Fit.rsquared),
        "adj_r_squared": float(salaryRegModel2Fit.rsquared_adj)
    },

    "sampleData": {
        "dataForTransfer": salaryRegModel2SampleData
    }
}
salaryRegModel2FitMetadata

{'modelName': 'salaryRegModel2Fit',
 'version': '1.0.1',
 'params': {'const': 44.09932687151595,
  'GPA': 6.71092896850212,
  'MIS': 5.325020933077331,
  'Statistics': 5.534983950443611,
  'misXStatistics1': 3.491456565315527},
 'score': 0.8095004820146132,
 'modelLibraray': 'sm.OLS',
 'libraryMetadata': {'pvalues': {'const': 2.1308984053354338e-47,
   'GPA': 2.206820882598934e-22,
   'MIS': 3.224529718282964e-11,
   'Statistics': 2.2590353916504234e-12,
   'misXStatistics1': 0.00423393188469119},
  'r_squared': 0.8095004820146132,
  'adj_r_squared': 0.8028744118238171},
 'sampleData': {'dataForTransfer': {'GPA': [3.53, 3.04, 3.65],
   'MIS': [1, 1, 0],
   'Statistics': [0, 1, 0]}}}

In [138]:
registerAJrjModel(salaryRegModel2Fit, salaryRegModel2FitMetadata)

✅ Uploaded encrypted ZIP to s3://jasonjafariml/salaryRegModel2Fit__1.0.1.pkl.zip


{'_id': '683fe44435856819138785c6',
 'modelName': 'salaryRegModel2Fit',
 'version': '1.0.1',
 'params': {'const': 44.09932687151595,
  'GPA': 6.71092896850212,
  'MIS': 5.325020933077331,
  'Statistics': 5.534983950443611,
  'misXStatistics1': 3.491456565315527},
 'score': 0.8095004820146132,
 'modelLibraray': 'sm.OLS',
 'libraryMetadata': {'pvalues': {'const': 2.1308984053354338e-47,
   'GPA': 2.206820882598934e-22,
   'MIS': 3.224529718282964e-11,
   'Statistics': 2.2590353916504234e-12,
   'misXStatistics1': 0.00423393188469119},
  'r_squared': 0.8095004820146132,
  'adj_r_squared': 0.8028744118238171},
 'sampleData': {'dataForTransfer': {'GPA': [3.53, 3.04, 3.65],
   'MIS': [1, 1, 0],
   'Statistics': [0, 1, 0]}},
 'modelType': 'model',
 'keepLastOnly': False,
 'modelSizeBytes': 23989,
 'zippedModelSizeBytes': 9130,
 's3Url': 'jasonjafariml/salaryRegModel2Fit__1.0.1.pkl.zip',
 'createdAt': '2025-06-04T06:14:28.909Z',
 'updatedAt': '2025-06-04T06:14:28.909Z'}

In [139]:
sallaryMisDf['misXGpa'] = sallaryMisDf.apply(lambda row: row['MIS'] * row['GPA'], axis=1)

sallaryMisDf


,Salary,GPA,MIS,Statistics,misXStatistics,misXStatistics1,misXGpa
0,72,3.53,1,0,0,0.0,3.53
1,66,2.86,1,0,0,0.0,2.86
2,72,3.69,0,0,0,0.0,0.00
3,63,3.24,0,0,0,0.0,0.00
4,65,3.21,0,0,0,0.0,0.00
...,...,...,...,...,...,...,...
115,66,3.27,0,0,0,0.0,0.00
116,63,2.86,1,0,0,0.0,2.86
117,78,3.04,1,1,1,1.0,3.04
118,64,2.99,0,0,0,0.0,0.00


In [140]:
salaryRegModel3 = sm.OLS(
  sallaryMisDf["Salary"],
  sm.add_constant(
      sallaryMisDf[[
          "GPA",
          "MIS",
          "Statistics",
          "misXGpa"
      ]]
  )
)

In [141]:
salaryRegModel3Fit = salaryRegModel3.fit()

In [142]:
print(salaryRegModel3Fit.summary())

                            OLS Regression Results                            
Dep. Variable:                 Salary   R-squared:                       0.795
Model:                            OLS   Adj. R-squared:                  0.788
Method:                 Least Squares   F-statistic:                     111.8
Date:                Wed, 04 Jun 2025   Prob (F-statistic):           1.11e-38
Time:                        02:14:29   Log-Likelihood:                -300.91
No. Observations:                 120   AIC:                             611.8
Df Residuals:                     115   BIC:                             625.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         44.1653      2.307     19.142      0.0

In [143]:
salaryRegModel3Fit = salaryRegModel3.fit()

In [144]:
print(salaryRegModel3Fit.summary())

                            OLS Regression Results                            
Dep. Variable:                 Salary   R-squared:                       0.795
Model:                            OLS   Adj. R-squared:                  0.788
Method:                 Least Squares   F-statistic:                     111.8
Date:                Wed, 04 Jun 2025   Prob (F-statistic):           1.11e-38
Time:                        02:14:29   Log-Likelihood:                -300.91
No. Observations:                 120   AIC:                             611.8
Df Residuals:                     115   BIC:                             625.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         44.1653      2.307     19.142      0.0

In [145]:
sallaryMisDf['statisticsXGpa'] = sallaryMisDf.apply(lambda row: row['Statistics'] * row['GPA'], axis=1)

sallaryMisDf

,Salary,GPA,MIS,Statistics,misXStatistics,misXStatistics1,misXGpa,statisticsXGpa
0,72,3.53,1,0,0,0.0,3.53,0.00
1,66,2.86,1,0,0,0.0,2.86,0.00
2,72,3.69,0,0,0,0.0,0.00,0.00
3,63,3.24,0,0,0,0.0,0.00,0.00
4,65,3.21,0,0,0,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...
115,66,3.27,0,0,0,0.0,0.00,0.00
116,63,2.86,1,0,0,0.0,2.86,0.00
117,78,3.04,1,1,1,1.0,3.04,3.04
118,64,2.99,0,0,0,0.0,0.00,0.00


In [146]:
salaryRegModel4 = sm.OLS(
  sallaryMisDf["Salary"],
  sm.add_constant(
      sallaryMisDf[[
          "GPA",
          "MIS",
          "Statistics",
          "statisticsXGpa"
      ]]
  )
)

In [147]:
salaryRegModel4Fit = salaryRegModel4.fit()

In [148]:
print(salaryRegModel4Fit.summary())

                            OLS Regression Results                            
Dep. Variable:                 Salary   R-squared:                       0.803
Model:                            OLS   Adj. R-squared:                  0.796
Method:                 Least Squares   F-statistic:                     116.9
Date:                Wed, 04 Jun 2025   Prob (F-statistic):           1.44e-39
Time:                        02:14:30   Log-Likelihood:                -298.78
No. Observations:                 120   AIC:                             607.6
Df Residuals:                     115   BIC:                             621.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             41.2856      2.267     18.

In [149]:
salaryRegModel5 = sm.OLS(
  sallaryMisDf["Salary"],
  sm.add_constant(
      sallaryMisDf[[
          "GPA",
          "MIS",
          "Statistics",
          "misXStatistics1",
          "misXGpa",
          "statisticsXGpa"
      ]]
  )
)

In [150]:
salaryRegModel5Fit = salaryRegModel5.fit()

In [151]:
print(salaryRegModel5Fit.summary())

                            OLS Regression Results                            
Dep. Variable:                 Salary   R-squared:                       0.815
Model:                            OLS   Adj. R-squared:                  0.805
Method:                 Least Squares   F-statistic:                     83.09
Date:                Wed, 04 Jun 2025   Prob (F-statistic):           4.15e-39
Time:                        02:14:30   Log-Likelihood:                -294.81
No. Observations:                 120   AIC:                             603.6
Df Residuals:                     113   BIC:                             623.1
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              41.7092      2.481     

In [152]:
salaryRegModel6 = sm.OLS(
  sallaryMisDf["Salary"],
  sm.add_constant(
      sallaryMisDf[[
          "GPA",
          # "MIS",
          "Statistics",
          "misXStatistics1",
          # "misXGpa",
          # "statisticsXGpa"
      ]]
  )
)

In [153]:
salaryRegModel6Fit = salaryRegModel6.fit()

In [154]:
print(salaryRegModel6Fit.summary())

                            OLS Regression Results                            
Dep. Variable:                 Salary   R-squared:                       0.720
Model:                            OLS   Adj. R-squared:                  0.713
Method:                 Least Squares   F-statistic:                     99.51
Date:                Wed, 04 Jun 2025   Prob (F-statistic):           6.12e-32
Time:                        02:14:31   Log-Likelihood:                -319.71
No. Observations:                 120   AIC:                             647.4
Df Residuals:                     116   BIC:                             658.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              45.7118      2.159     

## AFTER AREFULLY LOOKUP WE RELIZED  salaryRegModel4Fit SO LETS KEEP IT AS THE BEST

In [155]:
def salaryRegModel4Transformer(dataForTransfer = None):
    import pandas as pd
    import statsmodels.api as sm
    dfTransformer = pd.DataFrame(dataForTransfer)
    dfTransformer['statisticsXGpa'] = dfTransformer.apply(lambda row: row['Statistics'] * row['GPA'], axis=1)
    dfTransformer = sm.add_constant(dfTransformer[['GPA', 'MIS', 'Statistics', 'statisticsXGpa']])
    return dfTransformer

In [156]:
salaryRegModel4Fit.transformer = salaryRegModel4Transformer

In [157]:
salaryRegModel4Fit.mainPredictor = partial(salaryRegModel1MainPredictor, salaryRegModel4Fit)

In [158]:
test4 = salaryRegModel4Fit.transformer(salaryRegModel2SampleData)
test4

,const,GPA,MIS,Statistics,statisticsXGpa
0,1.0,3.53,1,0,0.00
1,1.0,3.04,1,1,3.04
2,1.0,3.65,0,0,0.00


In [159]:
salaryRegModel4Fit.mainPredictor(test4)

0    74.239878
1    77.909709
2    68.597785
dtype: float64

In [160]:
salaryRegModel4FitMetadata = {
    "modelName":f"salaryRegModel4Fit",
    "version":"1.0.1",
    "params": salaryRegModel4Fit.params.to_dict(),
    "score": float(salaryRegModel4Fit.rsquared),
    "modelLibraray": 'sm.OLS',
    "libraryMetadata": {
        "pvalues": salaryRegModel4Fit.pvalues.to_dict(),
        "r_squared": float(salaryRegModel4Fit.rsquared),
        "adj_r_squared": float(salaryRegModel4Fit.rsquared_adj)
    },

    "sampleData": {
        "dataForTransfer": salaryRegModel2SampleData
    }
}
salaryRegModel4FitMetadata

{'modelName': 'salaryRegModel4Fit',
 'version': '1.0.1',
 'params': {'const': 41.28560173035845,
  'GPA': 7.48278983439064,
  'MIS': 6.540027773861193,
  'Statistics': 14.598814781667048,
  'statisticsXGpa': -2.388952874805371},
 'score': 0.8025731076983917,
 'modelLibraray': 'sm.OLS',
 'libraryMetadata': {'pvalues': {'const': 1.1066906219868264e-35,
   'GPA': 6.696417260882898e-19,
   'MIS': 6.060691170759518e-20,
   'Statistics': 0.00027652830741481307,
   'statisticsXGpa': 0.0431204582314954},
  'r_squared': 0.8025731076983917,
  'adj_r_squared': 0.7957060853574662},
 'sampleData': {'dataForTransfer': {'GPA': [3.53, 3.04, 3.65],
   'MIS': [1, 1, 0],
   'Statistics': [0, 1, 0]}}}

In [161]:
registerAJrjModel(salaryRegModel4Fit, salaryRegModel4FitMetadata)

✅ Uploaded encrypted ZIP to s3://jasonjafariml/salaryRegModel4Fit__1.0.1.pkl.zip


{'_id': '683fe44a35856819138785c7',
 'modelName': 'salaryRegModel4Fit',
 'version': '1.0.1',
 'params': {'const': 41.28560173035845,
  'GPA': 7.48278983439064,
  'MIS': 6.540027773861193,
  'Statistics': 14.598814781667048,
  'statisticsXGpa': -2.388952874805371},
 'score': 0.8025731076983917,
 'modelLibraray': 'sm.OLS',
 'libraryMetadata': {'pvalues': {'const': 1.1066906219868264e-35,
   'GPA': 6.696417260882898e-19,
   'MIS': 6.060691170759518e-20,
   'Statistics': 0.00027652830741481307,
   'statisticsXGpa': 0.0431204582314954},
  'r_squared': 0.8025731076983917,
  'adj_r_squared': 0.7957060853574662},
 'sampleData': {'dataForTransfer': {'GPA': [3.53, 3.04, 3.65],
   'MIS': [1, 1, 0],
   'Statistics': [0, 1, 0]}},
 'modelType': 'model',
 'keepLastOnly': False,
 'modelSizeBytes': 23989,
 'zippedModelSizeBytes': 9205,
 's3Url': 'jasonjafariml/salaryRegModel4Fit__1.0.1.pkl.zip',
 'createdAt': '2025-06-04T06:14:34.376Z',
 'updatedAt': '2025-06-04T06:14:34.376Z'}

In [162]:
sallaryMisDf


,Salary,GPA,MIS,Statistics,misXStatistics,misXStatistics1,misXGpa,statisticsXGpa
0,72,3.53,1,0,0,0.0,3.53,0.00
1,66,2.86,1,0,0,0.0,2.86,0.00
2,72,3.69,0,0,0,0.0,0.00,0.00
3,63,3.24,0,0,0,0.0,0.00,0.00
4,65,3.21,0,0,0,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...
115,66,3.27,0,0,0,0.0,0.00,0.00
116,63,2.86,1,0,0,0.0,2.86,0.00
117,78,3.04,1,1,1,1.0,3.04,3.04
118,64,2.99,0,0,0,0.0,0.00,0.00
